In [15]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error #平方绝对误差
from sklearn.metrics import r2_score#R square
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [16]:
feature_names = pd.read_excel("主要特征变量.xlsx", na_values=np.nan)

In [17]:
feature_names_list = list(feature_names['主要特征变量'])

In [18]:
feature_names_list

['S-ZORB.CAL_H2.PV',
 'S-ZORB.FC_2801.PV',
 'S-ZORB.FT_5101.PV',
 'S-ZORB.FT_9001.PV',
 'S-ZORB.LC_5001.PV',
 'S-ZORB.LC_5101.PV',
 'S-ZORB.PC_2105.PV',
 'S-ZORB.PC_5101.PV',
 'S-ZORB.PDT_2104.PV',
 'S-ZORB.PT_2101.PV',
 'S-ZORB.PT_2801.PV',
 'S-ZORB.TC_5005.PV',
 'S-ZORB.TE_2005.PV',
 'S-ZORB.TE_2103.PV',
 'S-ZORB.TE_2301.PV',
 'S-ZORB.TE_5102.PV',
 'S-ZORB.TE_5202.PV',
 'S-ZORB.TE_9001.PV',
 '待生吸附剂性质：S',
 '待生吸附剂性质：焦炭',
 '原料性质：饱和烃',
 '原料性质：硫含量',
 '原料性质：密度',
 '原料性质：烯烃',
 '原料性质：辛烷值',
 '原料性质：溴值',
 '再生吸附剂性质：S',
 '再生吸附剂性质：焦炭']

In [19]:
data_325 = pd.read_excel("3-数据清洗后325样本数据.xlsx", na_values=np.nan)

In [20]:
data_325.columns[:30]

Index(['样本编号', '时间', 'RON损失', '原料性质：硫含量', '原料性质：辛烷值', '原料性质：饱和烃', '原料性质：烯烃',
       '原料性质：芳烃', '原料性质：溴值', '原料性质：密度', '产品性质：硫含量', '产品性质：辛烷值', '待生吸附剂性质：焦炭',
       '待生吸附剂性质：S', '再生吸附剂性质：焦炭', '再生吸附剂性质：S', 'S-ZORB.CAL_H2.PV',
       'S-ZORB.PDI_2102.PV', 'S-ZORB.PT_2801.PV', 'S-ZORB.FC_2801.PV',
       'S-ZORB.TE_2103.PV', 'S-ZORB.TE_2005.PV', 'S-ZORB.PT_2101.PV',
       'S-ZORB.PDT_2104.PV', 'S-ZORB.TE_2301.PV', 'S-ZORB.PT_2301.PV',
       'S-ZORB.PC_2105.PV', 'S-ZORB.PC_5101.PV', 'S-ZORB.TC_5005.PV',
       'S-ZORB.LC_5001.PV'],
      dtype='object')

In [21]:
feature_names_list.append('产品性质：辛烷值')
feature_names_list.append('RON损失')

In [22]:
data_325 = data_325[ feature_names_list ]

In [23]:
data_325.shape

(325, 30)

In [24]:
data_325.head()

,S-ZORB.CAL_H2.PV,S-ZORB.FC_2801.PV,S-ZORB.FT_5101.PV,S-ZORB.FT_9001.PV,S-ZORB.LC_5001.PV,S-ZORB.LC_5101.PV,S-ZORB.PC_2105.PV,S-ZORB.PC_5101.PV,S-ZORB.PDT_2104.PV,S-ZORB.PT_2101.PV,...,原料性质：饱和烃,原料性质：硫含量,原料性质：密度,原料性质：烯烃,原料性质：辛烷值,原料性质：溴值,再生吸附剂性质：S,再生吸附剂性质：焦炭,产品性质：辛烷值,RON损失
0,0.276318,647.752380,771.817045,479.093870,55.011075,49.916519,4.980083,0.652138,101.290580,2.393991,...,53.230000,188.0,726.085714,24.400000,90.6,61.487143,5.98,1.84,89.22,1.38
1,0.277127,651.821920,678.940915,470.983570,54.996157,49.955209,4.979703,0.652110,101.544925,2.394113,...,52.300000,169.0,731.300000,26.400000,90.5,61.880000,4.38,0.55,89.32,1.18
2,0.277013,650.747150,563.586215,433.017815,55.021892,50.215176,4.955568,0.652395,101.618320,2.394545,...,52.300000,177.0,729.614286,26.314286,90.7,61.722857,5.82,1.89,89.32,1.38
3,0.277222,652.669055,681.970820,440.395785,55.041748,49.877925,4.990756,0.652089,105.029760,2.392854,...,52.300000,159.0,725.400000,26.100000,90.4,61.330000,4.67,0.98,89.02,1.38
4,0.277310,649.340920,715.346780,411.576765,55.026846,49.900563,4.973214,0.652213,105.937455,2.392715,...,52.242857,173.0,725.428571,26.671429,89.6,61.332857,4.52,0.83,88.32,1.28


In [29]:
X = data_325[ data_325.columns[:28] ]
y_ron = data_325[ data_325.columns[28] ]
y_ron_loss = data_325[ data_325.columns[29] ]

In [30]:
X.head()

,S-ZORB.CAL_H2.PV,S-ZORB.FC_2801.PV,S-ZORB.FT_5101.PV,S-ZORB.FT_9001.PV,S-ZORB.LC_5001.PV,S-ZORB.LC_5101.PV,S-ZORB.PC_2105.PV,S-ZORB.PC_5101.PV,S-ZORB.PDT_2104.PV,S-ZORB.PT_2101.PV,...,待生吸附剂性质：S,待生吸附剂性质：焦炭,原料性质：饱和烃,原料性质：硫含量,原料性质：密度,原料性质：烯烃,原料性质：辛烷值,原料性质：溴值,再生吸附剂性质：S,再生吸附剂性质：焦炭
0,0.276318,647.752380,771.817045,479.093870,55.011075,49.916519,4.980083,0.652138,101.290580,2.393991,...,7.30,2.32,53.230000,188.0,726.085714,24.400000,90.6,61.487143,5.98,1.84
1,0.277127,651.821920,678.940915,470.983570,54.996157,49.955209,4.979703,0.652110,101.544925,2.394113,...,7.34,2.37,52.300000,169.0,731.300000,26.400000,90.5,61.880000,4.38,0.55
2,0.277013,650.747150,563.586215,433.017815,55.021892,50.215176,4.955568,0.652395,101.618320,2.394545,...,7.27,2.43,52.300000,177.0,729.614286,26.314286,90.7,61.722857,5.82,1.89
3,0.277222,652.669055,681.970820,440.395785,55.041748,49.877925,4.990756,0.652089,105.029760,2.392854,...,7.35,3.08,52.300000,159.0,725.400000,26.100000,90.4,61.330000,4.67,0.98
4,0.277310,649.340920,715.346780,411.576765,55.026846,49.900563,4.973214,0.652213,105.937455,2.392715,...,6.58,2.45,52.242857,173.0,725.428571,26.671429,89.6,61.332857,4.52,0.83


In [31]:
x_train_ron, x_test_ron, y_train_ron, y_test_ron = train_test_split(X, y_ron, test_size=0.3, random_state=1)
x_train_ron_loss, x_test_ron_loss, y_train_ron_loss, y_test_ron_loss = train_test_split(X, y_ron_loss, test_size=0.3, random_state=1)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

# ron model 1

In [43]:
model_RFR = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=666)
model_RFR.fit(x_train_ron, y_train_ron)
y_ron_predict_1 = model_RFR.predict(x_test_ron)
model_fearture = model_RFR.feature_importances_
mse_score = MSE(y_test_ron, y_ron_predict_1)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron, y_ron_predict_1)
rr_score = r2_score(y_test_ron, y_ron_predict_1)
print("随机森林回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

随机森林回归模型
MSE_SCORE =  0.05831773209724128
RMSE_SCORE =  0.2414906459829061
R_Square =  0.9406813823471101


# ron model 2

In [44]:
model_XGBR = XGBRegressor(learning_rate=0.1, max_depth=5, min_child_weight=6, n_estimators=50, random_state=666)
model_XGBR.fit(x_train_ron, y_train_ron)
y_ron_predict_2 = model_XGBR.predict(x_test_ron)
model_fearture = model_XGBR.feature_importances_
mse_score = MSE(y_test_ron, y_ron_predict_2)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron, y_ron_predict_2)
rr_score = r2_score(y_test_ron, y_ron_predict_2)
print("极端梯度提升回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

极端梯度提升回归模型
MSE_SCORE =  0.3061509637548889
RMSE_SCORE =  0.5533091032640697
R_Square =  0.6885946810695835


# ron model 3

In [45]:
model_AdaBR = AdaBoostRegressor(learning_rate=0.1, n_estimators=50, random_state=666)
model_AdaBR.fit(x_train_ron, y_train_ron)
y_ron_predict_3 = model_AdaBR.predict(x_test_ron)
model_fearture = model_AdaBR.feature_importances_
mse_score = MSE(y_test_ron, y_ron_predict_3)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron, y_ron_predict_3)
rr_score = r2_score(y_test_ron, y_ron_predict_3)
print("AdaBoost回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

AdaBoost回归模型
MSE_SCORE =  0.06320141472866121
RMSE_SCORE =  0.2513989155280134
R_Square =  0.9357138828862563


# ron model blending

In [46]:
y_blending = (y_ron_predict_1 + y_ron_predict_3 + y_ron_predict_3)/3.0

In [47]:
mse_score = MSE(y_test_ron, y_blending)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron, y_blending)
rr_score = r2_score(y_test_ron, y_blending)
print("三种模型融合")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

三种模型融合
MSE_SCORE =  0.059575923317088955
RMSE_SCORE =  0.24408179636566296
R_Square =  0.9394015972591043


In [72]:
x_test_ron_df = pd.DataFrame(x_test_ron['原料性质：辛烷值'])

In [73]:
x_test_ron_df = x_test_ron_df.reset_index(drop=True)

In [74]:
y_pre_ron = pd.DataFrame(y_blending)

In [75]:
y_pre_ron.columns = ['产品性质：辛烷值']
y_pre_ron = y_pre_ron.reset_index(drop=True)

In [76]:
y_ron_blending = x_test_ron_df["原料性质：辛烷值"] - y_pre_ron['产品性质：辛烷值']

# ron_loss model blending

In [79]:
mse_score = MSE(y_test_ron_loss, y_ron_blending)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron_loss, y_ron_blending)
rr_score = r2_score(y_test_ron_loss, y_ron_blending)
print("三种模型融合")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)

三种模型融合
MSE_SCORE =  0.059575923317088955
RMSE_SCORE =  0.24408179636566296


# ron_loss model 1

In [48]:
model_RFR = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=666)
model_RFR.fit(x_train_ron_loss, y_train_ron_loss)
y_ron_loss_predict_1 = model_RFR.predict(x_test_ron_loss)
model_fearture = model_RFR.feature_importances_
mse_score = MSE(y_test_ron_loss, y_ron_loss_predict_1)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron_loss, y_ron_loss_predict_1)
rr_score = r2_score(y_test_ron_loss, y_ron_loss_predict_1)
print("随机森林回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

随机森林回归模型
MSE_SCORE =  0.037328547730127795
RMSE_SCORE =  0.19320597229414985
R_Square =  0.14599618334122277


# ron_loss model 2

In [49]:
model_XGBR = XGBRegressor(learning_rate=0.1, max_depth=5, min_child_weight=6, n_estimators=50, random_state=666)
model_XGBR.fit(x_train_ron_loss, y_train_ron_loss)
y_ron_loss_predict_2 = model_XGBR.predict(x_test_ron_loss)
model_fearture = model_XGBR.feature_importances_
mse_score = MSE(y_test_ron_loss, y_ron_loss_predict_2)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron_loss, y_ron_loss_predict_2)
rr_score = r2_score(y_test_ron_loss, y_ron_loss_predict_2)
print("极端梯度提升回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

极端梯度提升回归模型
MSE_SCORE =  0.03904035855916872
RMSE_SCORE =  0.19758633191384653
R_Square =  0.10683331550163389


# ron_loss model 3

In [51]:
model_AdaBR = AdaBoostRegressor(learning_rate=0.1, n_estimators=50, random_state=666)
model_AdaBR.fit(x_train_ron_loss, y_train_ron_loss)
y_ron_loss_predict_3 = model_AdaBR.predict(x_test_ron_loss)
model_fearture = model_AdaBR.feature_importances_
mse_score = MSE(y_test_ron_loss, y_ron_loss_predict_3)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron_loss, y_ron_loss_predict_3)
rr_score = r2_score(y_test_ron_loss, y_ron_loss_predict_3)
print("AdaBoost回归模型")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)
print("R_Square = ", rr_score)

AdaBoost回归模型
MSE_SCORE =  0.040146344438551015
RMSE_SCORE =  0.20036552707127794
R_Square =  0.08153053198614035


# ron_loss model blending

In [52]:
y_ron_loss_blending = (y_ron_loss_predict_1 + y_ron_loss_predict_2 + y_ron_loss_predict_3)/3.0

In [80]:
mse_score = MSE(y_test_ron_loss, y_ron_loss_blending)
rmse_score = np.sqrt(mse_score)
mae_score = mean_absolute_error(y_test_ron_loss, y_ron_loss_blending)
rr_score = r2_score(y_test_ron_loss, y_ron_loss_blending)
print("三种模型融合")
print("MSE_SCORE = ", mse_score)
print("RMSE_SCORE = ", rmse_score)

三种模型融合
MSE_SCORE =  0.037526332262169226
RMSE_SCORE =  0.19371714498765777
